# 04 - Data Engineering: ETL and Storage Types

---

## What the Chapter Says

The chapter defines **Data Preparation = Data Engineering + Feature Engineering**.

This notebook covers **Data Engineering fundamentals**:

1. **Data Sources**: who collected, clean, trusted, user-generated vs system-generated
2. **Data Storage Overview**
3. **Database Types**:
   - SQL (MySQL, PostgreSQL)
   - NoSQL: key/value (Redis, DynamoDB), column-based (Cassandra, HBase), graph (Neo4J), document (MongoDB, CouchDB)
4. **ETL**: Extract, Transform (cleanse/map/format), Load (db/files/data warehouse)

---

## Meta Interview Signal

| Level | Expectations |
|-------|-------------|
| **E5** | Understands ETL pipeline. Can choose appropriate database type for use case. Knows structured vs unstructured data. |
| **E6** | Discusses data quality at scale. Designs for data freshness SLAs. Considers data lineage and versioning. Proposes hybrid storage solutions. |

---

## Data Preparation Pipeline Diagram (Chapter Figure)

The chapter shows this pipeline:

```
Data Sources → Data Engineering → Feature Engineering → Prepared Features
```

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import numpy as np

fig, ax = plt.subplots(figsize=(14, 4))
ax.axis('off')
ax.set_title('Data Preparation Pipeline (Chapter Figure)', fontsize=14, fontweight='bold')

# Pipeline boxes
boxes = [
    ('Data\nSources', 1, '#BBDEFB'),
    ('Data\nEngineering', 4.5, '#FFF9C4'),
    ('Feature\nEngineering', 8, '#C8E6C9'),
    ('Prepared\nFeatures', 11.5, '#E1BEE7'),
]

for (label, x, color) in boxes:
    rect = mpatches.FancyBboxPatch((x, 1), 2.5, 2, boxstyle='round,pad=0.1',
                                    facecolor=color, edgecolor='black', linewidth=2)
    ax.add_patch(rect)
    ax.text(x + 1.25, 2, label, ha='center', va='center', fontsize=12, fontweight='bold')

# Arrows
for x in [3.5, 7, 10.5]:
    ax.annotate('', xy=(x + 1, 2), xytext=(x, 2),
               arrowprops=dict(arrowstyle='->', color='black', lw=2))

# Labels
ax.text(2.25, 0.5, 'Logs, DBs,\nAPIs, Files', ha='center', fontsize=9, style='italic')
ax.text(5.75, 0.5, 'ETL: Extract,\nTransform, Load', ha='center', fontsize=9, style='italic')
ax.text(9.25, 0.5, 'Scaling, Encoding,\nImputation', ha='center', fontsize=9, style='italic')
ax.text(12.75, 0.5, 'Model-ready\nDataset', ha='center', fontsize=9, style='italic')

ax.set_xlim(0, 15)
ax.set_ylim(0, 4)
plt.tight_layout()
plt.show()

---

## E1) Data Sources (Chapter Content)

The chapter asks these questions about data sources:
- **Who collected** the data?
- Is it **clean**?
- Is it **trusted**?
- Is it **user-generated** vs **system-generated**?

In [ ]:
# Data source analysis framework
data_sources = pd.DataFrame({
    'Source Type': [
        'User-Generated',
        'User-Generated',
        'System-Generated',
        'System-Generated',
        'Third-Party'
    ],
    'Example': [
        'Posts, comments, reviews',
        'Profile information',
        'Click logs, view events',
        'Server logs, errors',
        'External APIs, purchased data'
    ],
    'Clean?': [
        'Often noisy, typos, spam',
        'Moderate - users may lie',
        'Usually clean, structured',
        'Clean but verbose',
        'Varies widely'
    ],
    'Trusted?': [
        'Low - adversarial users',
        'Moderate',
        'High - we control collection',
        'High',
        'Depends on vendor'
    ],
    'Interview Consideration': [
        'Need content moderation, spam filtering',
        'Validate critical fields',
        'Primary source for ML labels',
        'Good for debugging, monitoring',
        'Data contracts, freshness SLAs'
    ]
})

print("="*100)
print("DATA SOURCES ANALYSIS (Chapter Framework)")
print("="*100)
print(data_sources.to_string(index=False))

In [ ]:
# Simulate different data sources for a feed ranking system
np.random.seed(42)

# User-generated data (noisy)
n_posts = 1000
user_generated = pd.DataFrame({
    'post_id': range(n_posts),
    'text': ['Sample post text ' + str(i) for i in range(n_posts)],
    'author_id': np.random.randint(1, 101, n_posts),
    # User-generated often has issues:
    'has_typos': np.random.choice([True, False], n_posts, p=[0.15, 0.85]),
    'is_spam': np.random.choice([True, False], n_posts, p=[0.05, 0.95]),
    'missing_fields': np.random.choice([True, False], n_posts, p=[0.1, 0.9]),
})

print("USER-GENERATED DATA (Posts)")
print(f"Total posts: {len(user_generated)}")
print(f"Posts with typos: {user_generated['has_typos'].sum()} ({user_generated['has_typos'].mean()*100:.1f}%)")
print(f"Spam posts: {user_generated['is_spam'].sum()} ({user_generated['is_spam'].mean()*100:.1f}%)")
print(f"Posts with missing fields: {user_generated['missing_fields'].sum()}")

In [ ]:
# System-generated data (clean)
n_events = 10000
system_generated = pd.DataFrame({
    'event_id': range(n_events),
    'user_id': np.random.randint(1, 1001, n_events),
    'post_id': np.random.randint(0, n_posts, n_events),
    'action': np.random.choice(['view', 'like', 'share', 'comment'], n_events, p=[0.7, 0.15, 0.05, 0.1]),
    'timestamp': pd.date_range('2024-01-01', periods=n_events, freq='1min'),
    'device': np.random.choice(['iOS', 'Android', 'Web'], n_events, p=[0.4, 0.4, 0.2]),
})

print("\nSYSTEM-GENERATED DATA (Event Logs)")
print(f"Total events: {len(system_generated)}")
print(f"Missing values: {system_generated.isnull().sum().sum()}")
print(f"\nAction distribution:")
print(system_generated['action'].value_counts())

---

## E2) Database Types (Chapter Content)

The chapter lists these database types:

In [ ]:
# Database types from chapter
db_types = pd.DataFrame({
    'Category': [
        'SQL',
        'NoSQL - Key/Value',
        'NoSQL - Column-based',
        'NoSQL - Graph',
        'NoSQL - Document'
    ],
    'Examples': [
        'MySQL, PostgreSQL',
        'Redis, DynamoDB',
        'Cassandra, HBase',
        'Neo4J',
        'MongoDB, CouchDB'
    ],
    'Best For': [
        'Structured data with relationships, transactions',
        'Caching, session storage, real-time features',
        'Time-series, write-heavy workloads',
        'Social networks, recommendations',
        'Flexible schemas, JSON-like data'
    ],
    'ML Use Case': [
        'User profiles, product catalog',
        'Feature store (online), real-time predictions',
        'Event logs, training data at scale',
        'Friend recommendations, knowledge graphs',
        'Unstructured content, embeddings'
    ]
})

print("="*100)
print("DATABASE TYPES (from Chapter)")
print("="*100)
print(db_types.to_string(index=False))

In [ ]:
# Visual diagram of database types
fig, ax = plt.subplots(figsize=(14, 7))
ax.axis('off')
ax.set_title('Database Types Overview (Chapter)', fontsize=14, fontweight='bold')

# Main categories
# SQL
rect = mpatches.FancyBboxPatch((1, 5), 3, 1.5, boxstyle='round,pad=0.1',
                                facecolor='#BBDEFB', edgecolor='black', linewidth=2)
ax.add_patch(rect)
ax.text(2.5, 5.75, 'SQL', ha='center', va='center', fontsize=12, fontweight='bold')

# SQL examples
for i, db in enumerate(['MySQL', 'PostgreSQL']):
    rect = mpatches.FancyBboxPatch((0.5 + i*2, 3), 1.8, 1, boxstyle='round,pad=0.1',
                                    facecolor='#E3F2FD', edgecolor='black', linewidth=1)
    ax.add_patch(rect)
    ax.text(1.4 + i*2, 3.5, db, ha='center', va='center', fontsize=10)
    ax.annotate('', xy=(1.4 + i*2, 4), xytext=(2.5, 5),
               arrowprops=dict(arrowstyle='->', color='gray', lw=1))

# NoSQL
rect = mpatches.FancyBboxPatch((6, 5), 8, 1.5, boxstyle='round,pad=0.1',
                                facecolor='#C8E6C9', edgecolor='black', linewidth=2)
ax.add_patch(rect)
ax.text(10, 5.75, 'NoSQL', ha='center', va='center', fontsize=12, fontweight='bold')

# NoSQL subcategories
nosql_types = [
    ('Key/Value', 6.5, ['Redis', 'DynamoDB']),
    ('Column-based', 9, ['Cassandra', 'HBase']),
    ('Graph', 11.5, ['Neo4J']),
    ('Document', 13.5, ['MongoDB', 'CouchDB']),
]

for (subtype, x, examples) in nosql_types:
    rect = mpatches.FancyBboxPatch((x-0.7, 3), 1.8, 1, boxstyle='round,pad=0.1',
                                    facecolor='#E8F5E9', edgecolor='black', linewidth=1.5)
    ax.add_patch(rect)
    ax.text(x+0.2, 3.5, subtype, ha='center', va='center', fontsize=9, fontweight='bold')
    ax.annotate('', xy=(x+0.2, 4), xytext=(10, 5),
               arrowprops=dict(arrowstyle='->', color='gray', lw=1))
    
    # Examples
    for j, ex in enumerate(examples):
        rect = mpatches.FancyBboxPatch((x-0.5+j*0.9, 1), 0.8, 0.8, boxstyle='round,pad=0.05',
                                        facecolor='#F1F8E9', edgecolor='black', linewidth=1)
        ax.add_patch(rect)
        ax.text(x-0.1+j*0.9, 1.4, ex, ha='center', va='center', fontsize=7)
        ax.annotate('', xy=(x-0.1+j*0.9, 1.8), xytext=(x+0.2, 3),
                   arrowprops=dict(arrowstyle='->', color='lightgray', lw=0.5))

ax.set_xlim(0, 16)
ax.set_ylim(0, 7.5)
plt.tight_layout()
plt.show()

---

## E3) Structured vs Unstructured Data (Chapter Table)

The chapter provides this comparison table:

In [ ]:
# Chapter's structured vs unstructured table
data_comparison = pd.DataFrame({
    'Aspect': [
        'Schema',
        'Search',
        'Storage',
        'Examples',
        'ML Models'
    ],
    'Structured': [
        'Predefined schema',
        'Easier to search',
        'Relational DB, many NoSQL, data warehouses',
        'Dates, phone numbers, credit cards, addresses, names',
        'Traditional ML (logistic regression, trees, etc.)'
    ],
    'Unstructured': [
        'No schema',
        'Harder to search',
        'NoSQL, data lakes',
        'Text, audio, images, videos',
        'Deep learning spans both'
    ]
})

print("="*90)
print("STRUCTURED vs UNSTRUCTURED DATA (Chapter Table)")
print("="*90)
print(data_comparison.to_string(index=False))

In [ ]:
# Data types breakdown from chapter
print("\n" + "="*60)
print("DATA TYPES HIERARCHY (Chapter)")
print("="*60)

hierarchy = """
DATA TYPES
├── STRUCTURED
│   ├── Numerical
│   │   ├── Discrete (count of items, number of clicks)
│   │   └── Continuous (price, temperature, watch time)
│   └── Categorical
│       ├── Ordinal (rating: low/medium/high, education level)
│       └── Nominal (color, country, device type)
│
└── UNSTRUCTURED
    ├── Text (posts, comments, reviews)
    ├── Audio (voice messages, podcasts)
    ├── Image (photos, screenshots)
    └── Video (reels, stories)
"""
print(hierarchy)

In [ ]:
# Simulate structured data
np.random.seed(42)
n = 500

structured_data = pd.DataFrame({
    # Numerical - Discrete
    'num_clicks': np.random.poisson(5, n),
    'item_count': np.random.randint(1, 20, n),
    
    # Numerical - Continuous
    'watch_time_sec': np.random.exponential(120, n),
    'price_usd': np.random.uniform(10, 500, n),
    
    # Categorical - Ordinal
    'rating': np.random.choice(['low', 'medium', 'high'], n, p=[0.2, 0.5, 0.3]),
    
    # Categorical - Nominal
    'device': np.random.choice(['iOS', 'Android', 'Web'], n),
    'country': np.random.choice(['US', 'UK', 'IN', 'BR', 'DE'], n),
})

print("STRUCTURED DATA EXAMPLE")
print("="*60)
print(structured_data.head(10))
print(f"\nData types:")
print(structured_data.dtypes)

---

## E4) ETL: Extract, Transform, Load (Chapter Content)

In [ ]:
# ETL stages from chapter
etl_stages = pd.DataFrame({
    'Stage': ['Extract', 'Transform', 'Load'],
    'Description': [
        'Pull data from various sources',
        'Cleanse, map, and format data',
        'Store in target destination'
    ],
    'Operations': [
        'API calls, database queries, file reads',
        'Data cleansing, schema mapping, format conversion',
        'Database insert, file write, data warehouse load'
    ],
    'Targets': [
        'Source systems (DBs, APIs, files)',
        'In-memory processing',
        'Database, files, data warehouse'
    ]
})

print("="*80)
print("ETL PIPELINE (Chapter Content)")
print("="*80)
print(etl_stages.to_string(index=False))

In [ ]:
# Visual ETL diagram
fig, ax = plt.subplots(figsize=(14, 5))
ax.axis('off')
ax.set_title('ETL Pipeline (Chapter)', fontsize=14, fontweight='bold')

# Sources
sources = ['APIs', 'Databases', 'Files', 'Streams']
for i, src in enumerate(sources):
    rect = mpatches.FancyBboxPatch((0.5, 3.5 - i*1), 1.5, 0.8, boxstyle='round,pad=0.1',
                                    facecolor='#BBDEFB', edgecolor='black', linewidth=1)
    ax.add_patch(rect)
    ax.text(1.25, 3.9 - i*1, src, ha='center', va='center', fontsize=9)

# Extract box
rect = mpatches.FancyBboxPatch((3, 1.5), 2, 2.5, boxstyle='round,pad=0.1',
                                facecolor='#FFF9C4', edgecolor='black', linewidth=2)
ax.add_patch(rect)
ax.text(4, 2.75, 'EXTRACT', ha='center', va='center', fontsize=12, fontweight='bold')

# Arrows to Extract
for i in range(4):
    ax.annotate('', xy=(3, 2.75), xytext=(2, 3.9 - i*1),
               arrowprops=dict(arrowstyle='->', color='gray', lw=1))

# Transform box
rect = mpatches.FancyBboxPatch((6, 1.5), 2.5, 2.5, boxstyle='round,pad=0.1',
                                facecolor='#FFCCBC', edgecolor='black', linewidth=2)
ax.add_patch(rect)
ax.text(7.25, 3.2, 'TRANSFORM', ha='center', va='center', fontsize=12, fontweight='bold')
ax.text(7.25, 2.6, '• Cleanse', ha='center', va='center', fontsize=9)
ax.text(7.25, 2.2, '• Map', ha='center', va='center', fontsize=9)
ax.text(7.25, 1.8, '• Format', ha='center', va='center', fontsize=9)

ax.annotate('', xy=(6, 2.75), xytext=(5, 2.75),
           arrowprops=dict(arrowstyle='->', color='black', lw=2))

# Load box
rect = mpatches.FancyBboxPatch((9.5, 1.5), 2, 2.5, boxstyle='round,pad=0.1',
                                facecolor='#C8E6C9', edgecolor='black', linewidth=2)
ax.add_patch(rect)
ax.text(10.5, 2.75, 'LOAD', ha='center', va='center', fontsize=12, fontweight='bold')

ax.annotate('', xy=(9.5, 2.75), xytext=(8.5, 2.75),
           arrowprops=dict(arrowstyle='->', color='black', lw=2))

# Destinations
destinations = ['Database', 'Files', 'Data Warehouse']
for i, dst in enumerate(destinations):
    rect = mpatches.FancyBboxPatch((12.5, 3 - i*1), 2, 0.8, boxstyle='round,pad=0.1',
                                    facecolor='#E1BEE7', edgecolor='black', linewidth=1)
    ax.add_patch(rect)
    ax.text(13.5, 3.4 - i*1, dst, ha='center', va='center', fontsize=9)
    ax.annotate('', xy=(12.5, 3.4 - i*1), xytext=(11.5, 2.75),
               arrowprops=dict(arrowstyle='->', color='gray', lw=1))

ax.set_xlim(0, 15.5)
ax.set_ylim(0, 5)
plt.tight_layout()
plt.show()

---

## Hands-On: Simulated ETL Pipeline

In [ ]:
# Simulate a realistic ETL pipeline for feed ranking
np.random.seed(42)

class ETLPipeline:
    """Simulates ETL for ML data preparation"""
    
    def __init__(self):
        self.raw_data = None
        self.transformed_data = None
        self.stats = {'extracted': 0, 'transformed': 0, 'loaded': 0}
    
    def extract(self, sources):
        """EXTRACT: Pull data from multiple sources"""
        print("\n" + "="*60)
        print("EXTRACT STAGE")
        print("="*60)
        
        all_data = []
        for source_name, data in sources.items():
            print(f"  Extracting from {source_name}: {len(data)} records")
            data['source'] = source_name
            all_data.append(data)
        
        self.raw_data = pd.concat(all_data, ignore_index=True)
        self.stats['extracted'] = len(self.raw_data)
        print(f"\n  Total extracted: {self.stats['extracted']} records")
        return self
    
    def transform(self):
        """TRANSFORM: Cleanse, map, format"""
        print("\n" + "="*60)
        print("TRANSFORM STAGE")
        print("="*60)
        
        df = self.raw_data.copy()
        initial_count = len(df)
        
        # 1. Cleanse: Remove nulls and duplicates
        null_count = df.isnull().any(axis=1).sum()
        df = df.dropna()
        print(f"  [Cleanse] Removed {null_count} rows with nulls")
        
        dup_count = df.duplicated().sum()
        df = df.drop_duplicates()
        print(f"  [Cleanse] Removed {dup_count} duplicate rows")
        
        # 2. Map: Standardize column names and values
        if 'timestamp' in df.columns:
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            print("  [Map] Converted timestamp to datetime")
        
        if 'action' in df.columns:
            df['action'] = df['action'].str.lower()
            print("  [Map] Standardized action values to lowercase")
        
        # 3. Format: Create derived fields
        if 'timestamp' in df.columns:
            df['hour_of_day'] = df['timestamp'].dt.hour
            df['day_of_week'] = df['timestamp'].dt.dayofweek
            print("  [Format] Created hour_of_day and day_of_week")
        
        self.transformed_data = df
        self.stats['transformed'] = len(df)
        
        print(f"\n  Transformed: {initial_count} → {self.stats['transformed']} records")
        print(f"  Dropped: {initial_count - self.stats['transformed']} records")
        return self
    
    def load(self, destination='data_warehouse'):
        """LOAD: Store to destination"""
        print("\n" + "="*60)
        print("LOAD STAGE")
        print("="*60)
        
        self.stats['loaded'] = len(self.transformed_data)
        print(f"  Loaded {self.stats['loaded']} records to {destination}")
        print(f"\n  Final schema:")
        print(f"  {list(self.transformed_data.columns)}")
        return self.transformed_data
    
    def summary(self):
        """Print pipeline summary"""
        print("\n" + "="*60)
        print("ETL PIPELINE SUMMARY")
        print("="*60)
        print(f"  Extracted: {self.stats['extracted']}")
        print(f"  Transformed: {self.stats['transformed']}")
        print(f"  Loaded: {self.stats['loaded']}")
        print(f"  Data loss: {self.stats['extracted'] - self.stats['loaded']} records "
              f"({(1 - self.stats['loaded']/self.stats['extracted'])*100:.1f}%)")

In [ ]:
# Create sample source data
np.random.seed(42)

# Source 1: Click logs
click_logs = pd.DataFrame({
    'user_id': np.random.randint(1, 1001, 5000),
    'post_id': np.random.randint(1, 2001, 5000),
    'action': np.random.choice(['view', 'like', 'share', None], 5000, p=[0.65, 0.2, 0.1, 0.05]),
    'timestamp': pd.date_range('2024-01-01', periods=5000, freq='2min'),
})

# Source 2: Mobile app events
mobile_events = pd.DataFrame({
    'user_id': np.random.randint(1, 1001, 3000),
    'post_id': np.random.randint(1, 2001, 3000),
    'action': np.random.choice(['VIEW', 'LIKE', 'COMMENT'], 3000, p=[0.6, 0.25, 0.15]),
    'timestamp': pd.date_range('2024-01-01', periods=3000, freq='3min'),
})

# Run ETL
pipeline = ETLPipeline()
result = (pipeline
    .extract({'click_logs': click_logs, 'mobile_events': mobile_events})
    .transform()
    .load('feature_store'))

pipeline.summary()

In [ ]:
# Show transformed data sample
print("\nTransformed Data Sample:")
print(result.head(10))

---

## Tradeoffs (Chapter-Aligned)

| Tradeoff | Discussion | Interview Signal |
|----------|------------|------------------|
| **SQL vs NoSQL** | ACID compliance vs horizontal scaling | E5: Knows when each applies. E6: Proposes hybrid (SQL for users, NoSQL for events) |
| **Batch vs Streaming ETL** | Throughput vs freshness | E5: Understands difference. E6: Designs for different freshness SLAs |
| **Data Warehouse vs Data Lake** | Schema-on-write vs schema-on-read | E5: Can explain both. E6: Discusses cost/flexibility tradeoffs at scale |
| **Denormalization** | Query speed vs storage/consistency | E5: Knows tradeoff. E6: Discusses specific denormalization for ML features |

---

## Meta Interview Signal (Detailed)

### E5 Answer Expectations

- Understands ETL stages and what happens in each
- Can choose appropriate database type for a use case
- Knows difference between structured and unstructured data
- Can explain data sources: user-generated vs system-generated

### E6 Additions

- **Data quality at scale**: "At Meta scale, we need data validation pipelines that catch schema drift before it breaks models"
- **Freshness SLAs**: "Real-time features need sub-second freshness, while daily aggregates can have 24h delay"
- **Data lineage**: "We track data lineage so when a feature breaks, we can trace back to the source"
- **Hybrid storage**: "User profiles in SQL for ACID, event logs in Cassandra for write throughput, features in Redis for low-latency serving"

---

## Interview Drills

### Drill 1: Database Selection
For each use case, choose the appropriate database type and justify:
- User profiles with relationships (friends)
- Real-time feature lookup during inference
- Storing 1B daily events for training
- Flexible content storage (posts with varying fields)

### Drill 2: ETL Design
Design an ETL pipeline for a video recommendation system:
- What sources would you extract from?
- What transformations are needed?
- Where would you load the results?

### Drill 3: Data Source Analysis
For a harmful content detection system:
- List 3 user-generated data sources
- List 3 system-generated data sources
- Which is more trustworthy and why?

### Drill 4: Structured vs Unstructured
Classify these data types and explain how you'd process each:
- User age
- Post text
- Profile photo
- Number of followers
- Account creation date

### Drill 5: Scale Discussion (E6)
Discuss how ETL changes at Meta scale:
- 1B daily active users
- 100B events per day
- Sub-second freshness requirements for some features